In [123]:
import os
from dotenv import load_dotenv
load_dotenv()

python-dotenv could not parse statement starting at line 2


True

In [124]:
# Tool and LLM imports
from langchain_community.tools import TavilySearchResults
from langchain_groq import ChatGroq

In [125]:
# Message types for managing the conversation history
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage

In [126]:
# 1. Define the Tool: Tavily Search
tavily_tool = TavilySearchResults(max_results=3, name="tavily_search_results_json")
tavily_tool.description = "A powerful search engine for finding real-time, up-to-date information on the internet. Use this for any questions about current events, news, weather, or topics beyond your internal knowledge."

# A dictionary to easily look up our tool by its name
tools = {tavily_tool.name: tavily_tool}

# 2. Set up the LLM and bind the tool
llm = ChatGroq(model="llama-3.1-8b-instant")
llm_with_tools = llm.bind_tools([tavily_tool]) # We pass the tool object itself here

print("Tool and LLM initialized.")

Tool and LLM initialized.


In [127]:
def run_agent(user_query: str):
    """
    Manually runs the agent loop without LangGraph.
    """
    # The system message is the CRITICAL instruction that guides the LLM's behavior.
    system_message = SystemMessage(
        content="""You are a helpful assistant. You have access to a powerful internet search tool called 'tavily_search_results_json'.

        - Use this tool to answer any questions that require real-time information, such as news, weather, or recent events.
        - For general conversation or questions you can answer from your own knowledge, respond directly without using the tool."""
    )
    
    # Start the conversation history
    messages = [system_message, HumanMessage(content=user_query)]
    
    while True:
        print("... Calling LLM ...")
        
        # 1. Call the LLM with the current history
        ai_response = llm_with_tools.invoke(messages)
        messages.append(ai_response)
        
        # 2. Check if the LLM wants to call a tool
        if not ai_response.tool_calls:
            # If not, the conversation is over. Break the loop.
            print("... LLM provided final answer. Ending loop. ...")
            break
            
        # 3. If a tool call is present, execute it
        print("... LLM wants to use a tool ...")
        for tool_call in ai_response.tool_calls:
            tool_to_run = tools[tool_call['name']]
            tool_output = tool_to_run.invoke(tool_call['args'])
            
            # 4. Add the tool's output back to the conversation history
            messages.append(ToolMessage(content=str(tool_output), tool_call_id=tool_call['id']))

    return messages

print(" Manual agent loop function defined.")

 Manual agent loop function defined.


In [128]:
# A query that will force the agent to use its tool
proof_query = "What is the current temperature in Greater Noida?"

# Run our manual agent
final_messages = run_agent(proof_query)

print("\n--- Agent Execution Trace ---")
for message in final_messages:
    # Skip the system message for a cleaner output
    if isinstance(message, SystemMessage):
        continue
    message.pretty_print()
print("\n--- End of Trace ---")

... Calling LLM ...
... LLM wants to use a tool ...
... Calling LLM ...
... LLM provided final answer. Ending loop. ...

--- Agent Execution Trace ---
================================ Human Message =================================

What is the current temperature in Greater Noida?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (95gfbnmy1)
 Call ID: 95gfbnmy1
  Args:
    query: current temperature in Greater Noida
================================= Tool Message =================================

[{'title': 'Greater Noida, Uttar Pradesh, India Weather Forecast | AccuWeather', 'url': 'https://www.accuweather.com/en/in/greater-noida/3143979/weather-forecast/3143979', 'content': "# Greater Noida, Uttar Pradesh\n\nGreater Noida\n\nUttar Pradesh\n\n## Around the Globe\n\nAround the Globe\n\n### Hurricane Tracker\n\n### Severe Weather\n\n### Radar & Maps\n\n### News & Features\n\n### Astronomy\n\n### Business\n\n### Cli

In [129]:
#pip install langgraph langchain langchain_groq langchain_community tavily-python google-search-results python-dotenv

In [130]:
# A simple query that should not use a tool
no_tool_query = "Hey, what's up?"

# Run our manual agent
final_messages = run_agent(no_tool_query)

print("\n--- Agent Execution Trace ---")
for message in final_messages:
    if isinstance(message, SystemMessage):
        continue
    message.pretty_print()
print("\n--- End of Trace ---")

... Calling LLM ...
... LLM provided final answer. Ending loop. ...

--- Agent Execution Trace ---
================================ Human Message =================================

Hey, what's up?
================================== Ai Message ==================================

I'm doing well, thanks for asking. It's nice to chat with you. How about you, how's your day going so far?

--- End of Trace ---


In [131]:
# A simple query that should not use a tool
tool_query = "summarise eveyrthing what happened in the latest apple event launch in bullet points "

# Run our manual agent
final_messages = run_agent(tool_query)

print("\n--- Agent Execution Trace ---")
for message in final_messages:
    if isinstance(message, SystemMessage):
        continue
    message.pretty_print()
print("\n--- End of Trace ---")

... Calling LLM ...
... LLM wants to use a tool ...
... Calling LLM ...
... LLM provided final answer. Ending loop. ...

--- Agent Execution Trace ---
================================ Human Message =================================

summarise eveyrthing what happened in the latest apple event launch in bullet points 
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (qd457wxmz)
 Call ID: qd457wxmz
  Args:
    query: latest apple event launch summary
================================= Tool Message =================================

[{'title': 'iPhone 17 Air, AirPods Pro 3, Apple Watch Series 11 and more', 'url': 'https://www.cnbc.com/2025/09/09/live-updates-apple-event-iphone17-airpods-ios26.html', 'content': '## Apple CEO Tim Cook talks design in opening remarks\n\nApple CEO Tim Cook kicked off the launch event with a short speech about how the company thinks about design, saying that it\'s "at the core of everythin

In [132]:
# A simple query that should not use a tool
tool_query = " Give 10 most important news that happened in last 24 hrs "

# Run our manual agent
final_messages = run_agent(tool_query)

print("\n--- Agent Execution Trace ---")
for message in final_messages:
    if isinstance(message, SystemMessage):
        continue
    message.pretty_print()
print("\n--- End of Trace ---")

... Calling LLM ...
... LLM wants to use a tool ...
... Calling LLM ...
... LLM provided final answer. Ending loop. ...

--- Agent Execution Trace ---
================================ Human Message =================================

 Give 10 most important news that happened in last 24 hrs 
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (z4cg3zd42)
 Call ID: z4cg3zd42
  Args:
    query: top news in last 24 hours
================================= Tool Message =================================

[{'title': "CBS News | Breaking news, top stories & today's latest headlines", 'url': 'https://www.cbsnews.com/', 'content': "the mail.  18H agoImage 88: 0328-techwatch-2794223-640x360.jpg  #### How AI powered robots are helping small farms From labor shortages to environmental impacts, farmers are looking to AI to help revolutionize the agriculture industry. One California startup, Farm-ng, is tapping into the power of AI 

In [133]:
final_messages[-1].content

'Based on the search results, here are the top 10 most important news from the last 24 hours:\n\n1. Connecticut to ban first cousin marriages, becoming the 26th state to do so.\n2. Former President Trump\'s administration announces a new rule that believes acetaminophen is linked to autism risk, but the maker of Tylenol and medical experts have pushed back.\n3. American captain Keegan Bradley defended the decision to play in a golf tournament after being asked about the optics of it.\n4. Sean "Diddy" Combs asks for release on time served on prostitution conviction.\n5. The Ultimate Culinary Celebration Hits the Big Apple.\n6. Russian airstrikes kill three and wound two in Zaporizhzhia.\n7. Moldova detains 74 over alleged Russian plot to stir up unrest.\n8. Putin: Russia to respect nuclear arms limits with US for one more year.\n9. Thousands of workers across Italy go on strike in solidarity with Gaza.\n10. Trump honours \'martyr\' Charlie Kirk at memorial attended by thousands.\n\nNote

In [135]:
# A simple query that should not use a tool
query = " write a poem on the moon landing from the sun's perspective"

# Run our manual agent
final_messages = run_agent(query)

print("\n--- Agent Execution Trace ---")
for message in final_messages:
    if isinstance(message, SystemMessage):
        continue
    message.pretty_print()
print("\n--- End of Trace ---")

... Calling LLM ...
... LLM provided final answer. Ending loop. ...

--- Agent Execution Trace ---
================================ Human Message =================================

 write a poem on the moon landing from the sun's perspective
================================== Ai Message ==================================

As I shine my light upon the earth below
I see a momentous occasion, a feat to show
A giant leap for mankind, a dream to take
The moon landing, a story to make

I watch as Apollo 11 takes to the sky
A spacecraft carrying brave men, with a mission to try
I see the eagles rising, with a fire in their soul
Their hearts ablaze with wonder, as they take their goal

I shine my rays upon the lunar surface bright
As Armstrong steps out, into the moon's pale light
He takes a step, and then another, with a stride so bold
A moment of history, forever to be told

I see the Earth from afar, a blue and green sphere
A beautiful sight, with a fragile atmosphere
The astronauts, a tiny